# How to use BusNet

In [1]:
# Imports all needed for BusNet4
!pip install geopandas

import networkx as nx
import pandas as pd
import geopandas
from datetime import datetime
import numpy as np
import sys
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
from IPython.display import clear_output
import pickle
import math
import time

import BusNet4 as bus

C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


BusNet4 imported


ERROR: Could not find a version that satisfies the requirement iython (from versions: none)
ERROR: No matching distribution found for iython


# Initialisation

Parsing the GTFS files can be slow (especially if you extract from larger data sources such as the UK Bus Open Data  https://www.gov.uk/guidance/find-and-use-bus-open-data ).  To get around this, BusNet once BusNet has parsed the GTFS data and created the graph, it saves the data structure in two pickle files:

 - <name>.graph.pickle
 - <name>.stops.pickle

You may then load the structure quickly from the cache to speed up queries.

## Initialisation without a cache

Assuming that you have the cache files:

- dundeeworking.graph.pickle
- dundeeworking.stops.pickle

You can initialise BusNet directly from the cache as below:

In [2]:
bus.setup(cache="dundeeworking")

Cache loaded
851


In [3]:
import json

# 
# START HERE!

# Create a bounding box for the area that we're interested in 
j = json.load(open('./dundee_boundaries.geojson'))    
j = j['features'][0]
coords = j['geometry']['coordinates'][0]
dundeeBoundary = []
for c in coords:
    dundeeBoundary.append(Point(c[1],c[0]))
    
# dundeeBoundary now contains a list of points that define Dundee. Only those services within that polygon will be extracted
    
    
bus.setup(cache="dundeeworking",validAgency = ["Ember","Stagecoach East Scotland","Moffat & Williamson","Xplore Dundee"],boundingPoly = dundeeBoundary)
# This creates the bus model
# If the cache is specified then...  if <name>.stops.pickle and <name>.graph.pickle exist in the working directory they will be 
# used to create the model.  Otherwise the other params will be used to create a new model from scratch based on GTFS data that should 
# be locaed in ./itm_all_gtfs/

# validAgency lists those gtfs agencies which should be included in the model
# boundingPoly defines the area to that the model will be built from
# for a route to be in the model it must be within the area AND operated by a valid agency


# NOTE:  building and caching models takes time (ie hours in some cases).  It is best to build once and then cache!



Cache loaded
851


In [ ]:
bus.setup(cache="dundeeworking")


In [4]:


# Test Dundee
start = (56.46356336480093, -3.0368739883858193) 
dest = (56.47163455953654, -3.0114212570733745)
r = bus.findPath(start,end=dest,walk=0.5)
# walk is the maximum dist that will be walked to/from the bus stop

print(r)
# r is a tuple:
# 1 status, should be 'found' if all has worked, otherwise an eror message
# 2 time of journey in mins (includes walking)
# 3 summary of journey, bus stop codes and services used 
# 4 verbose description of journey - used for debugging




Using end
('found', 37.41761186460506, ['start', '640011695', '10:Xplore Dundee:Broughty Ferry', '6400PT1174', 'end'], ['Journey:', '0 : start', 0, '1 : 640011695', 'James Arnott Drive', 8.692099103675641, '2 : 10:Xplore Dundee:Broughty Ferry', 29.024197869107738, '3 : 6400PT1174', 'Lochee By-Pass', 29.024197869107738, '4 : end', 37.41761186460506])


In [5]:

# City centre

# As an alternative to specifying an end point a polygon describing an area can be specified. This will find the quickest route 
# from the start to any stop within the area

city_centre=[]
city_centre.append(Point(56.45521456343855, -2.975808604)) 
city_centre.append(Point(56.46084193778188, -2.9768997916807534)) 
city_centre.append(Point(56.463990223978904, -2.967079106389851)) 
city_centre.append(Point(56.460239044693814, -2.9604107398342996)) 

r = bus.findPath(start,walk=0.5,centre=city_centre)
print(r)



Using centre
('found', 28.5623518523047, ['start', '6400S128', '73:Stagecoach East Scotland:Monifieth', '6400PT1380', 'end'], ['Journey:', '0 : start', 0, '1 : 6400S128', 'Earn Crescent', 6.468937856807386, '2 : 73:Stagecoach East Scotland:Monifieth', 20.168937856807386, '3 : 6400PT1380', 'West Marketgait', 20.168937856807386, '4 : end', 28.5623518523047])


In [ ]:
# Run a batch of tests to find routes to the city centre

points = []
points.append((56.46356336480093, -3.0368739883858193)) 
points.append((56.46926335285665, -3.0293204073095032)) 
points.append((56.47514192611533, -2.94623630343118)) 
points.append((56.45503723262541, -2.997048069439411)) 
points.append((56.4808300012964, -2.9709555409486974))
points.append((56.45921078022061, -3.0344702484589865))
points.append((56.46736684471463, -3.0049444925352846))
points.append((56.47115976627114, -2.936279943875513))
points.append((56.48007164052372, -3.0004812968723997))
points.append((56.47153903758696, -3.0389334441218714))
points.append((56.46888405880653, -2.9376532348487085))

for start in points:
    r = bus.findPath(start,centre=city_centre)
    print("Run :")
    print(r)